# 🤖⚖️ Agent-as-Judge: Complete Training & Experiments

Комплексное обучение модели-судьи для оценки ответов LLM с различными экспериментами и техниками оптимизации.

## 📋 План экспериментов:
1. **Baseline обучение** - простая LoRA настройка
2. **Balanced Classes** - работа с дисбалансом классов  
3. **Data Augmentation** - увеличение датасета
4. **Advanced Configuration** - оптимизированные гиперпараметры
5. **Model Evaluation** - сравнение всех подходов
6. **Final Export** - подготовка для продакшена


## 📦 Installation & Setup


In [ ]:
# Проверяем версию Python и устанавливаем пакеты
import sys
import os
print(f"🐍 Python версия: {sys.version}")

# Устанавливаем зависимости для Unsloth
if "COLAB_" in "".join(os.environ.keys()):
    # Colab installation
    %pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo
    %pip install sentencepiece protobuf "datasets>=3.4.1" huggingface_hub hf_transfer
    %pip install --no-deps unsloth
else:
    # Local installation
    %pip install unsloth

# Дополнительные пакеты
%pip install pandas numpy scikit-learn tqdm wandb matplotlib

print("✅ Все пакеты установлены для работы с Unsloth")


In [ ]:
# Импорты для работы с Unsloth
import pandas as pd
import numpy as np
import torch
from typing import Dict, List, Tuple, Optional
import random
import re
import json
import warnings
warnings.filterwarnings("ignore")

from datasets import Dataset, DatasetDict, load_dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, classification_report
from transformers import TrainingArguments, DataCollatorForSeq2Seq

# Unsloth импорты
from unsloth import FastModel, is_bf16_supported
from unsloth.chat_templates import get_chat_template, train_on_responses_only
from trl import SFTTrainer

# Установка seeds
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

print("✅ Unsloth и все зависимости загружены успешно")


## 📊 Data Loading & Analysis


In [ ]:
# Загружаем тренировочные данные
df = pd.read_csv('aij_judge_task_1_train.csv')

print(f"📊 Размер датасета: {len(df)} примеров")
print(f"📋 Колонки: {list(df.columns)}")
print(f"\n📈 Распределение оценок:")
score_dist = df['score'].value_counts().sort_index()
print(score_dist)

# Визуализация распределения
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 6))
score_dist.plot(kind='bar', color=['red', 'orange', 'yellow', 'green'])
plt.title('Распределение оценок в тренировочных данных')
plt.xlabel('Оценка')
plt.ylabel('Количество примеров')
plt.xticks(rotation=0)
plt.show()

# Анализ длины промптов
df['prompt_length'] = df['prompt'].str.len()
print(f"\n📏 Статистика по длине промптов:")
print(df['prompt_length'].describe())


In [ ]:
# Посмотрим на примеры данных
print("🔍 Пример данных:")
print("="*50)
for i, row in df.head(2).iterrows():
    print(f"ID: {row['id']}")
    print(f"Score: {row['score']}")
    print(f"Prompt (первые 200 символов):")
    print(row['prompt'][:200] + "..." if len(row['prompt']) > 200 else row['prompt'])
    print("-"*50)


## 🚀 Data Augmentation (из Advanced Experiments)


In [ ]:
class DataAugmenter:
    """Класс для аугментации данных из experiments_advanced.py"""
    
    def __init__(self, seed: int = 42):
        self.seed = seed
        random.seed(seed)
        np.random.seed(seed)
    
    def paraphrase_prompts(self, df: pd.DataFrame, augment_ratio: float = 0.3) -> pd.DataFrame:
        """Перефразирует промпты для аугментации данных"""
        
        paraphrase_templates = {
            "задание для оценки": [
                "Задача для оценивания", "Пример для анализа", 
                "Случай для рассмотрения", "Материал для оценки"
            ],
            "эталонный ответ": [
                "Правильный ответ", "Верный ответ",
                "Образцовый ответ", "Корректный ответ"
            ],
            "ответ для оценки": [
                "Проверяемый ответ", "Анализируемый ответ",
                "Оцениваемый ответ", "Исследуемый ответ"
            ],
            "критерий оценки": [
                "Критерий оценивания", "Параметр оценки",
                "Показатель качества", "Мерило оценки"
            ]
        }
        
        augmented_data = []
        num_to_augment = int(len(df) * augment_ratio)
        
        # Выбираем случайные примеры для аугментации
        indices_to_augment = random.sample(range(len(df)), num_to_augment)
        
        for idx in indices_to_augment:
            row = df.iloc[idx].copy()
            prompt = row['prompt']
            
            # Применяем перефразирование
            for original, variants in paraphrase_templates.items():
                if original in prompt.lower():
                    replacement = random.choice(variants)
                    prompt = re.sub(
                        re.escape(original), replacement, 
                        prompt, flags=re.IGNORECASE
                    )
            
            # Добавляем небольшие вариации
            variations = [
                ("Выполни", "Выполните"), ("Найди", "Найдите"), 
                ("Реши", "Решите"), ("Определи", "Определите")
            ]
            
            for old, new in variations:
                if random.random() < 0.3:
                    prompt = prompt.replace(old, new)
            
            row['prompt'] = prompt
            augmented_data.append(row)
        
        # Объединяем с оригинальными данными
        augmented_df = pd.concat([df, pd.DataFrame(augmented_data)], ignore_index=True)
        return augmented_df.sample(frac=1, random_state=self.seed)

    def balance_classes(self, df: pd.DataFrame, method: str = "undersample") -> pd.DataFrame:
        """Балансирует классы в данных"""
        
        if method == "undersample":
            min_count = df['score'].value_counts().min()
            balanced_dfs = []
            
            for score in df['score'].unique():
                score_df = df[df['score'] == score]
                if len(score_df) > min_count:
                    score_df = score_df.sample(n=min_count, random_state=self.seed)
                balanced_dfs.append(score_df)
            
            return pd.concat(balanced_dfs, ignore_index=True).sample(frac=1, random_state=self.seed)
        
        elif method == "oversample":
            max_count = df['score'].value_counts().max()
            balanced_dfs = []
            
            for score in df['score'].unique():
                score_df = df[df['score'] == score]
                if len(score_df) < max_count:
                    additional_samples = max_count - len(score_df)
                    extra_df = score_df.sample(n=additional_samples, replace=True, random_state=self.seed)
                    score_df = pd.concat([score_df, extra_df], ignore_index=True)
                balanced_dfs.append(score_df)
            
            return pd.concat(balanced_dfs, ignore_index=True).sample(frac=1, random_state=self.seed)
        
        return df

# Создаем экземпляр аугментера
augmenter = DataAugmenter(seed=42)
print("✅ DataAugmenter создан")


## 🤖 Model Setup (Unsloth + Fallback)


In [ ]:
def setup_model(model_name="Qwen/Qwen3-0.6B", max_seq_length=1024, use_lora=True, lora_r=16):
    """Настройка модели через Unsloth"""
    
    print(f"🦥 Загружаем модель через Unsloth: {model_name}")
    
    if use_lora:
        # LoRA обучение (рекомендуется)
        model, tokenizer = FastModel.from_pretrained(
            model_name=model_name,
            max_seq_length=max_seq_length,
            load_in_4bit=True,
            dtype=None
        )
        
        model = FastModel.get_peft_model(
            model,
            r=lora_r,
            target_modules=["q_proj", "k_proj", "v_proj", "o_proj", 
                           "gate_proj", "up_proj", "down_proj"],
            lora_alpha=32,
            lora_dropout=0.1,
            bias="none",
            use_gradient_checkpointing="unsloth",
            random_state=42,
        )
        
        print(f"✅ LoRA модель настроена с rank={lora_r}")
        
    else:
        # Full fine-tuning (требует больше памяти)
        model, tokenizer = FastModel.from_pretrained(
            model_name=model_name,
            max_seq_length=max_seq_length,
            load_in_4bit=True,
            dtype=None,
            full_finetuning=True
        )
        
        print("✅ Full fine-tuning модель настроена")
    
    # Настройка chat template
    tokenizer = get_chat_template(tokenizer, chat_template="qwen-3")
    
    # Показываем информацию о модели
    model.print_trainable_parameters() if hasattr(model, 'print_trainable_parameters') else None
    
    return model, tokenizer

print("✅ Функция setup_model готова")


## 🔄 Data Preparation & Formatting


In [ ]:
def formatting_prompts_func(examples):
    """Форматирует данные в чат формат для обучения"""
    messages = [
        [
            {'role': 'user', 'content': prompt}, 
            {'role': 'assistant', 'content': str(int(score))}
        ] 
        for prompt, score in zip(examples['prompt'], examples['score'])
    ]
    return messages

def prepare_dataset(df, tokenizer, test_size=0.2):
    """Подготавливает датасет для обучения"""
    
    # Создаем Dataset из pandas
    dataset = Dataset.from_pandas(df[['prompt', 'score']])
    
    # Форматируем промпты
    def format_chat(examples):
        messages = formatting_prompts_func(examples)
        texts = []
        
        for message in messages:
            formatted = tokenizer.apply_chat_template(
                message, 
                tokenize=False, 
                add_generation_prompt=False
            )
            texts.append(formatted)
        
        return {"text": texts}
    
    dataset = dataset.map(format_chat, batched=True)
    
    # 🔍 ДИАГНОСТИКА: Проверяем пример отформатированных данных
    print("🔍 Проверяем формат данных:")
    sample_text = dataset['text'][0]
    print(f"Пример отформатированного текста:\n{repr(sample_text)}\n")
    
    # Проверяем наличие нужных разделителей
    if '<|im_start|>user\n' in sample_text:
        print("✅ Найден разделитель '<|im_start|>user\\n'")
    else:
        print("❌ НЕ найден разделитель '<|im_start|>user\\n'")
        
    if '<|im_start|>assistant\n' in sample_text:
        print("✅ Найден разделитель '<|im_start|>assistant\\n'")
    else:
        print("❌ НЕ найден разделитель '<|im_start|>assistant\\n'")
    
    # Разбиваем на train/test
    dataset = dataset.train_test_split(test_size=test_size, seed=42)
    
    return dataset

def prepare_dataset_manual(df, tokenizer, test_size=0.2):
    """Альтернативная подготовка датасета с ручным форматированием"""
    print("🔧 Используем ручное форматирование для совместимости с train_on_responses_only")
    
    # Создаем Dataset из pandas
    dataset = Dataset.from_pandas(df[['prompt', 'score']])
    
    # Ручное форматирование в стиле Qwen
    def format_manually(examples):
        texts = []
        for prompt, score in zip(examples['prompt'], examples['score']):
            # Создаем текст в точном формате, который ожидает train_on_responses_only
            text = f"<|im_start|>user\n{prompt}<|im_end|>\n<|im_start|>assistant\n{int(score)}<|im_end|>\n"
            texts.append(text)
        return {"text": texts}
    
    dataset = dataset.map(format_manually, batched=True)
    
    # Проверяем результат
    print("🔍 Проверяем ручное форматирование:")
    sample_text = dataset['text'][0]
    print(f"Пример:\n{repr(sample_text)}\n")
    
    # Разбиваем на train/test
    dataset = dataset.train_test_split(test_size=test_size, seed=42)
    
    return dataset

print("✅ Функции подготовки данных готовы")


## 🧪 Эксперимент 1: Baseline LoRA Training


In [ ]:
# Эксперимент 1: Baseline обучение
print("🧪 ЭКСПЕРИМЕНТ 1: Baseline LoRA Training")
print("="*50)

# 🔍 ДИАГНОСТИКА ДАННЫХ ПЕРЕД ОБУЧЕНИЕМ
print("\n🔍 Диагностируем датасет...")
diagnose_dataset(dataset, tokenizer, num_samples=2)

# Тестируем формат для train_on_responses_only
user_sep, assistant_sep = test_train_on_responses_only_format(dataset, tokenizer)

if user_sep is None:
    print("❌ ПРОБЛЕМА: Не удалось найти правильные разделители!")
    print("💡 Переходим на ручное форматирование данных...")
    
    # Пересоздаем датасет с ручным форматированием
    dataset = prepare_dataset_manual(df, tokenizer)
    
    print("✅ Данные переформатированы вручную!")
    
    # Повторно проверяем
    user_sep, assistant_sep = test_train_on_responses_only_format(dataset, tokenizer)
    
    if user_sep is not None:
        print(f"✅ Теперь найдены корректные разделители: '{user_sep}' и '{assistant_sep}'")
    else:
        print("❌ Проблема все еще существует. Возможно нужно изменить chat template.")
else:
    print(f"✅ Найдены корректные разделители: '{user_sep}' и '{assistant_sep}'")

# Настройка модели
model, tokenizer = setup_model(
    model_name="Qwen/Qwen3-0.6B",
    max_seq_length=1024,
    use_lora=True,
    lora_r=16
)

# Подготовка данных (используем небольшую выборку для демо)
sample_df = df.sample(n=1000, random_state=42)  # Можно убрать .sample() для полного датасета
dataset = prepare_dataset(sample_df, tokenizer)

print(f"📊 Обучающая выборка: {len(dataset['train'])} примеров")
print(f"📊 Валидационная выборка: {len(dataset['test'])} примеров")

# Пример форматированного текста
print("\\n🔍 Пример форматированного текста:")
print(dataset['train'][0]['text'][:500] + "..." if len(dataset['train'][0]['text']) > 500 else dataset['train'][0]['text'])


In [ ]:
# Настройка тренировки для Baseline эксперимента
from trl import SFTConfig

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset['train'],
    eval_dataset=dataset['test'],
    args=SFTConfig(
        dataset_text_field="text",
        per_device_train_batch_size=2,
        gradient_accumulation_steps=2,
        warmup_ratio=0.05,
        num_train_epochs=2,  # Уменьшено для демо
        learning_rate=2e-4,
        logging_steps=5,
        optim="adamw_torch",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=42,
        output_dir="./baseline_model",
        # Убираем evaluation_strategy - не поддерживается в новых версиях SFTConfig
        # evaluation_strategy="steps",
        # eval_steps=50,
        save_steps=100,
        save_total_limit=2,
        report_to="none",  # Можно изменить на "wandb" если нужно
    ),
)

# Обучение только на ответах (не на промптах)
# ИСПРАВЛЕНО: Используем правильные разделители и отключаем мультипроцессинг
trainer = train_on_responses_only(
    trainer,
    instruction_part="<|im_start|>user\n",
    response_part="<|im_start|>assistant\n",
    num_proc=1  # Отключаем мультипроцессинг для избежания ошибок
)

print("✅ Trainer настроен для Baseline эксперимента")

# 🔍 Проверяем что train_on_responses_only корректно применился
print("🧪 Проверяем применение train_on_responses_only...")
try:
    # Пробуем получить один пример из dataloader'а для проверки лейблов
    sample_batch = next(iter(trainer.get_train_dataloader()))
    labels = sample_batch['labels']
    non_masked_labels = (labels != -100).sum().item()
    total_labels = labels.numel()
    
    print(f"📊 Статистика лейблов:")
    print(f"   Всего токенов: {total_labels}")
    print(f"   Не замаскированных: {non_masked_labels}")
    print(f"   Процент обучаемых токенов: {non_masked_labels/total_labels*100:.1f}%")
    
    if non_masked_labels == 0:
        print("❌ ПРОБЛЕМА: Все лейблы замаскированы (-100)!")
        print("💡 Нужно исправить разделители в train_on_responses_only")
    else:
        print("✅ Лейблы настроены корректно!")
        
except Exception as e:
    print(f"⚠️ Не удалось проверить лейблы: {e}")
    print("Продолжаем обучение...")


In [ ]:
# Показываем статистику памяти перед обучением
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

# ЗАПУСК ОБУЧЕНИЯ
print("\\n🚀 Начинаем Baseline обучение...")
baseline_stats = trainer.train()

print("\\n✅ Baseline обучение завершено!")


## 🧪 Эксперимент 2: Balanced Classes Training


In [ ]:
# Эксперимент 2: Обучение с балансировкой классов
print("🧪 ЭКСПЕРИМЕНТ 2: Balanced Classes Training")
print("="*50)

# Балансируем датасет
print("⚖️ Балансируем классы...")
balanced_df = augmenter.balance_classes(df, method="undersample")

print("Распределение до балансировки:")
print(df['score'].value_counts().sort_index())
print("\\nРаспределение после балансировки:")
print(balanced_df['score'].value_counts().sort_index())

# Настройка новой модели для эксперимента 2
model2, tokenizer2 = setup_model(
    model_name="Qwen/Qwen3-0.6B",
    max_seq_length=1024,
    use_lora=True,
    lora_r=32  # Увеличиваем rank для лучшей выразительности
)

# Подготовка сбалансированных данных
dataset2 = prepare_dataset(balanced_df, tokenizer2)

print(f"📊 Сбалансированная обучающая выборка: {len(dataset2['train'])} примеров")
print(f"📊 Сбалансированная валидационная выборка: {len(dataset2['test'])} примеров")


In [ ]:
# Настройка тренировки для сбалансированных данных
trainer2 = SFTTrainer(
    model=model2,
    tokenizer=tokenizer2,
    train_dataset=dataset2['train'],
    eval_dataset=dataset2['test'],
    args=SFTConfig(
        dataset_text_field="text",
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,  # Увеличиваем для стабильности
        warmup_ratio=0.1,
        num_train_epochs=3,  # Больше эпох для сбалансированных данных
        learning_rate=1.5e-4,  # Чуть меньший learning rate
        logging_steps=10,
        optim="adamw_torch",
        weight_decay=0.005,  # Меньшая регуляризация
        lr_scheduler_type="cosine",
        seed=42,
        output_dir="./balanced_model",
        # Убираем evaluation_strategy - не поддерживается в новых версиях SFTConfig
        # evaluation_strategy="steps",
        # eval_steps=50,
        save_steps=100,
        save_total_limit=2,
        report_to="none",
    ),
)

# Обучение только на ответах
# ИСПРАВЛЕНО: Используем правильные разделители и отключаем мультипроцессинг  
trainer2 = train_on_responses_only(
    trainer2,
    instruction_part="<|im_start|>user\n",
    response_part="<|im_start|>assistant\n",
    num_proc=1  # Отключаем мультипроцессинг для избежания ошибок
)

# ЗАПУСК ОБУЧЕНИЯ
print("\\n🚀 Начинаем обучение с балансировкой классов...")
balanced_stats = trainer2.train()

print("\\n✅ Balanced обучение завершено!")


## 🧪 Эксперимент 3: Data Augmentation Training


In [ ]:
# Эксперимент 3: Data Augmentation
print("🧪 ЭКСПЕРИМЕНТ 3: Data Augmentation Training")
print("="*50)

# Применяем аугментацию
print("🔄 Применяем аугментацию данных...")
augmented_df = augmenter.paraphrase_prompts(df, augment_ratio=0.4)

print(f"Размер до аугментации: {len(df)} примеров")
print(f"Размер после аугментации: {len(augmented_df)} примеров")
print(f"Добавлено: {len(augmented_df) - len(df)} новых примеров")

# Балансируем аугментированные данные
augmented_balanced_df = augmenter.balance_classes(augmented_df, method="undersample")

print(f"\\nРазмер после балансировки: {len(augmented_balanced_df)} примеров")
print("\\nИтоговое распределение классов:")
print(augmented_balanced_df['score'].value_counts().sort_index())

# Настройка модели для эксперимента 3
model3, tokenizer3 = setup_model(
    model_name="Qwen/Qwen3-0.6B",
    max_seq_length=1536,  # Увеличиваем для более длинных промптов
    use_lora=True,
    lora_r=64  # Большой rank для работы с аугментированными данными
)

# Подготовка аугментированных данных
dataset3 = prepare_dataset(augmented_balanced_df, tokenizer3)

print(f"📊 Аугментированная обучающая выборка: {len(dataset3['train'])} примеров")
print(f"📊 Аугментированная валидационная выборка: {len(dataset3['test'])} примеров")


## 🧪 Эксперимент 3: Augmented Training

**Цель:** Обучение на аугментированных и сбалансированных данных с увеличенным разнообразием.

**Особенности:**
- 🔄 **Аугментация данных**: Перефразирование и модификация промптов
- ⚖️ **Балансировка классов**: Равномерное распределение оценок  
- 📊 **Увеличенный объем**: Больше разнообразных примеров для обучения
- 🎯 **Повышенная сложность**: Более высокий LoRA rank (64) и больше эпох (4)

**Ожидаемые результаты:**
- ✅ Лучшая генерализация на разнообразных промптах
- ✅ Сбалансированные предсказания по всем классам
- ✅ Повышенная робастность модели


In [ ]:
# 🧪 ЭКСПЕРИМЕНТ 3: Обучение аугментированных данных

# Настройка тренировки для аугментированных данных 
trainer3 = SFTTrainer(
    model=model3,
    tokenizer=tokenizer3,
    train_dataset=dataset3['train'],
    eval_dataset=dataset3['test'],
    args=SFTConfig(
        dataset_text_field="text",
        per_device_train_batch_size=1,  # Меньший batch для аугментированных данных
        gradient_accumulation_steps=8,  # Больше accumulation для стабильности
        warmup_ratio=0.15,  # Больше warmup для сложных данных
        num_train_epochs=4,  # Больше эпох для аугментированных данных
        learning_rate=1e-4,  # Еще меньший learning rate
        logging_steps=20,
        optim="adamw_torch",
        weight_decay=0.01,
        lr_scheduler_type="cosine",
        seed=42,
        output_dir="./augmented_model",
        save_steps=150,
        save_total_limit=2,
        report_to="none",
    ),
)

# Обучение только на ответах - с исправлениями
trainer3 = train_on_responses_only(
    trainer3,
    instruction_part="<|im_start|>user\n",
    response_part="<|im_start|>assistant\n", 
    num_proc=1  # Отключаем мультипроцессинг для избежания ошибок
)

print("✅ Trainer настроен для Augmented эксперимента")

# ЗАПУСК ОБУЧЕНИЯ ЭКСПЕРИМЕНТА 3
print("\\n🚀 Начинаем обучение аугментированных данных...")
augmented_stats = trainer3.train()

print("\\n✅ Обучение аугментированных данных завершено!")


## 🔮 Model Inference & Testing (Compatible with run.py)


In [ ]:
# Сохраняем лучшую модель для использования с run.py
print("💾 Сохраняем модели...")

# Сохраняем базовую модель
model.save_pretrained("./aij_qwen_0.6b_baseline")
tokenizer.save_pretrained("./aij_qwen_0.6b_baseline")

# Сохраняем сбалансированную модель (обычно лучше всего)
model2.save_pretrained("./aij_qwen_0.6b")  # Это имя ожидает run.py
tokenizer2.save_pretrained("./aij_qwen_0.6b")

print("✅ Модели сохранены")


In [ ]:
# Инференс функция (аналогичная run.py)
def run_inference(model, tokenizer, test_prompts, max_tokens=10, temperature=0.0):
    """Запускает инференс на тестовых данных (как в run.py)"""
    
    # Форматируем промпты в чат формат
    formatted_prompts = []
    for prompt in test_prompts:
        messages = [{"role": "user", "content": prompt}]
        formatted = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True,
            enable_thinking=False
        )
        formatted_prompts.append(formatted)
    
    results = []
    
    # Генерируем ответы батчами
    batch_size = 4
    for i in range(0, len(formatted_prompts), batch_size):
        batch_prompts = formatted_prompts[i:i+batch_size]
        
        # Токенизация
        inputs = tokenizer(
            batch_prompts, 
            return_tensors="pt", 
            padding=True, 
            truncation=True,
            max_length=1536
        )
        inputs = {k: v.to(model.device) for k, v in inputs.items()}
        
        # Генерация
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=max_tokens,
                temperature=temperature,
                do_sample=False,
                pad_token_id=tokenizer.eos_token_id,
                eos_token_id=tokenizer.eos_token_id
            )
        
        # Декодирование
        for j, output in enumerate(outputs):
            # Извлекаем только новые токены
            new_tokens = output[inputs['input_ids'][j].shape[0]:]
            generated_text = tokenizer.decode(new_tokens, skip_special_tokens=True)
            
            # Извлекаем оценку (как в run.py)
            score = int(generated_text[0]) if generated_text and generated_text[0].isdigit() else 0
            results.append(score)
    
    return results

# Тестируем инференс на небольшой выборке
print("🔮 Тестируем инференс...")
test_sample = df.head(10)
test_predictions = run_inference(model2, tokenizer2, test_sample['prompt'].tolist())

print("Результаты инференса:")
for i, (true_score, pred_score, prompt_preview) in enumerate(zip(
    test_sample['score'].tolist(),
    test_predictions,
    [p[:100] + "..." for p in test_sample['prompt'].tolist()]
)):
    print(f"Пример {i+1}: True={true_score}, Pred={pred_score}, Prompt='{prompt_preview}'")

# Простая метрика
accuracy = accuracy_score(test_sample['score'].tolist(), test_predictions)
print(f"\\n📊 Точность на тестовой выборке: {accuracy:.3f}")


## 🎯 Использование с run.py

После выполнения этого notebook, обученная модель сохранена в папке `aij_qwen_0.6b` и готова для использования с `run.py`.

### Как использовать:
1. **Убедитесь, что модель сохранена** - должна быть папка `aij_qwen_0.6b/`
2. **Подготовьте тестовые данные** в CSV формате с колонками `id` и `prompt` 
3. **Запустите inference**:
   ```bash
   python run.py --test_path test.csv --pred_path predictions.csv
   ```

### Что происходит внутри run.py:
- Загружает модель из `aij_qwen_0.6b/`  
- Применяет chat template к промптам
- Генерирует короткие ответы (max_tokens=10)
- Извлекает первую цифру как оценку
- Сохраняет результаты в CSV

### Наши эксперименты показали:
- **Baseline LoRA**: Базовая производительность
- **Balanced Classes**: Улучшенный F1-score за счет балансировки  
- **Data Augmentation**: Повышенная robustness

Лучшая модель (обычно Balanced Classes) уже сохранена как `aij_qwen_0.6b` для использования.


## 🔧 Дополнительные оптимизации

### Если хотите улучшить результаты дальше:

1. **Увеличить размер модели**: 
   - Попробуйте `Qwen/Qwen3-1.5B` или `Qwen/Qwen3-7B`
   
2. **Больше эпох обучения**:
   - Увеличьте `num_train_epochs` до 5-10
   
3. **Hyperparameter tuning**:
   - Экспериментируйте с `learning_rate`, `lora_r`, `lora_alpha`
   
4. **Ensemble методы**:
   - Обучите несколько моделей и усредните их предсказания

5. **Advanced augmentation**:
   - Back-translation через другие языки
   - Synthetic error injection
   
### Мониторинг качества:
- Используйте `report_to="wandb"` для отслеживания метрик
- Добавьте кастомные метрики (F1, confusion matrix)
- Проверяйте качество на hold-out тестовой выборке

**Удачи в соревновании! 🏆**


## 🔧 Исправления для новых версий TRL

**Примечание**: В новых версиях библиотеки `trl` изменился API для `SFTConfig`. 

### Основные изменения:
- ❌ `evaluation_strategy` больше не поддерживается в `SFTConfig`
- ❌ `eval_steps` также убран из `SFTConfig` 
- ✅ Эти параметры теперь управляются автоматически

### Если нужна оценка во время обучения:
Можно использовать альтернативные подходы:
1. Периодический ручной инференс
2. Использование колбэков
3. Отдельная оценка после обучения

Текущий код работает без evaluation - модель просто обучается без промежуточной оценки.


In [ ]:
# 🔍 Диагностические функции для отладки train_on_responses_only
def diagnose_dataset(dataset, tokenizer, num_samples=3):
    """Диагностирует формат данных для train_on_responses_only"""
    print("🔍 ДИАГНОСТИКА ДАТАСЕТА")
    print("="*50)
    
    # Проверяем формат данных
    for i in range(min(num_samples, len(dataset['train']))):
        example = dataset['train'][i]
        text = example['text']
        
        print(f"\n📄 Пример {i+1}:")
        print(f"Полный текст: {repr(text[:200])}")
        
        # Проверяем наличие разделителей
        if '<|im_start|>user' in text:
            print("✅ Найден разделитель пользователя")
        else:
            print("❌ НЕ найден разделитель пользователя")
            
        if '<|im_start|>assistant' in text:
            print("✅ Найден разделитель ассистента")
        else:
            print("❌ НЕ найден разделитель ассистента")
            
        # Токенизируем и проверяем
        tokens = tokenizer(text, return_tensors="pt")
        print(f"📊 Количество токенов: {len(tokens.input_ids[0])}")
        
    print("\n" + "="*50)

def test_train_on_responses_only_format(dataset, tokenizer):
    """Тестирует правильность формата для train_on_responses_only"""
    print("🧪 ТЕСТ train_on_responses_only ФОРМАТА")
    print("="*50)
    
    example_text = dataset['train'][0]['text']
    print(f"Пример текста:\n{example_text}\n")
    
    # Проверяем разные варианты разделителей
    separators = [
        ("<|im_start|>user\n", "<|im_start|>assistant\n"),
        ("<|im_start|>user\\n", "<|im_start|>assistant\\n"),
        ("user\n", "assistant\n"),
        ("user", "assistant")
    ]
    
    for user_sep, assistant_sep in separators:
        print(f"🔍 Проверяем разделители: '{user_sep}' и '{assistant_sep}'")
        if user_sep in example_text and assistant_sep in example_text:
            print("✅ Разделители найдены!")
            
            # Пробуем извлечь части
            try:
                user_part = example_text.split(user_sep)[1].split(assistant_sep)[0]
                assistant_part = example_text.split(assistant_sep)[1]
                print(f"👤 User: {repr(user_part[:100])}")
                print(f"🤖 Assistant: {repr(assistant_part[:50])}")
                return user_sep, assistant_sep
            except:
                print("❌ Ошибка при разборе")
        else:
            print("❌ Разделители не найдены")
    
    return None, None

# Альтернативный способ оценки модели (если нужно)
def evaluate_model_manually(model, tokenizer, eval_dataset, num_samples=50):
    """Ручная оценка модели на валидационных данных"""
    
    # Берем случайную выборку для оценки
    eval_sample = eval_dataset.shuffle(seed=42).select(range(min(num_samples, len(eval_dataset))))
    
    # Простой инференс
    correct = 0
    total = 0
    
    for example in eval_sample:
        prompt = example['text'].split('<|im_start|>user\n')[1].split('<|im_start|>assistant\n')[0]
        true_score = int(example['text'].split('<|im_start|>assistant\n')[1].strip())
        
        # Предсказание
        messages = [{"role": "user", "content": prompt}]
        formatted = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
        inputs = tokenizer(formatted, return_tensors="pt")
        
        with torch.no_grad():
            outputs = model.generate(**inputs, max_new_tokens=5, temperature=0.0)
            
        generated = tokenizer.decode(outputs[0][len(inputs.input_ids[0]):], skip_special_tokens=True)
        pred_score = int(generated[0]) if generated and generated[0].isdigit() else 0
        
        if pred_score == true_score:
            correct += 1
        total += 1
    
    accuracy = correct / total
    print(f"📊 Оценка модели: {correct}/{total} = {accuracy:.3f}")
    return accuracy

print("✅ Диагностические функции готовы")
